In [ ]:
# create some tables in database system for sql practice 

In [1]:
# import necessary lybraries

import pandas as pd
import numpy as np


# import connection 

import sqlite3
conn = sqlite3.connect('pokemon.db')

cur  = conn.cursor()

In [4]:
cur.execute(""" create table emp(empno integer primary key,
ename text not null, job text,mgr integer,
hiredate numeric,sal integer,comm integer,deptno integer not null);""")


OperationalError: table emp already exists

In [7]:
cur.execute(""" create table dept(deptno integer primary key,
dname text not null, loc text);""")

In [22]:
cur.execute("""select * from dept;""")
cur.fetchall()

[(10, 'ACCOUNTING', 'NEW YORK'),
 (20, 'RESEARCH', 'DALLAS'),
 (30, 'SALES', 'CHICAGO'),
 (40, 'OPERATIONS', 'BOSTON')]

In [10]:
cur.execute("select * from emp;")
cur.fetchall()

[(7369, 'SMITH', 'CLERK', 7902, '17-DEC-80', 800, None, 20),
 (7499, 'ALLEN', 'SALESMAN', 7698, '20-FEB-81', 1600, 300, 30),
 (7521, 'WARD', 'SALESMAN', 7698, '22-FEB-81', 1250, 500, 30),
 (7566, 'JONES', 'MANAGER', 7839, '2-APR-81', 2975, None, 20),
 (7654, 'MARTIN', 'SALESMAN', 7839, '2-APR-81', 1250, 1400, 30),
 (7902, 'ford', 'ANALYST', 7566, '3-DEC-81', 3000, None, 20)]

In [11]:
cur.execute(""" update emp 
set ename = 'FORD'
where empno = 7902""")

In [9]:
# insert dummy values into emp table for practice 

cur.execute(""" insert into emp(empno,ename,job,mgr,hiredate,sal,deptno) 
values(7369,'SMITH','CLERK',7902,'17-DEC-80',800,20);""")

cur.execute(""" insert into emp(empno,ename,job,mgr,hiredate,sal,comm,deptno) 
values(7499,'ALLEN','SALESMAN',7698,'20-FEB-81',1600,300,30);""")

cur.execute(""" insert into emp(empno,ename,job,mgr,hiredate,sal,comm,deptno) 
values(7521,'WARD','SALESMAN',7698,'22-FEB-81',1250,500,30);""");

cur.execute(""" insert into emp(empno,ename,job,mgr,hiredate,sal,comm,deptno) 
values(7654,'MARTIN','SALESMAN',7839,'2-APR-81',1250,1400,30);""")

cur.execute(""" insert into emp(empno,ename,job,mgr,hiredate,sal,deptno) 
values(7566,'JONES','MANAGER',7839,'2-APR-81',2975,20);""");

cur.execute(""" insert into emp(empno,ename,job,mgr,hiredate,sal,deptno) 
values(7902,'ford','ANALYST',7566,'3-DEC-81',3000,20);""")

In [8]:
# insert dummy values into dept table for practices

cur.execute(""" insert into dept(deptno,dname,loc) values(10,'ACCOUNTING','NEW YORK')""");
cur.execute(""" insert into dept(deptno,dname,loc) values(20,'RESEARCH','DALLAS')""");
cur.execute(""" insert into dept(deptno,dname,loc) values(30,'SALES','CHICAGO')""");
cur.execute(""" insert into dept(deptno,dname,loc) values(40,'OPERATIONS','BOSTON')""");

In [12]:
# practice sub queries on emp and dept table 


cur.execute(""" SELECT * FROM emp 
WHERE mgr = (SELECT mgr FROM emp WHERE ename = 'JONES');
""")

df_sql1 = pd.DataFrame(cur.fetchall(),columns =[ item[0] for item in cur.description ])
df_sql1

,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7566,JONES,MANAGER,7839,2-APR-81,2975,NaN,20
1,7654,MARTIN,SALESMAN,7839,2-APR-81,1250,1400.0,30


In [61]:
# pairwise comparison

cur.execute(""" SELECT * FROM emp 
WHERE (deptno, mgr) = (SELECT deptno, mgr FROM emp WHERE ename = 'FORD');
""")

df_sql2 = pd.DataFrame(cur.fetchall(),columns = [ item[0] for item in cur.description ] )
df_sql2

,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7902,FORD,ANALYST,7566,3-DEC-81,3000,None,20


In [60]:
# non pairwise comparison

cur.execute(""" SELECT * FROM emp 
WHERE deptno = (SELECT deptno FROM emp WHERE ename='FORD') 
AND mgr = (SELECT mgr FROM emp WHERE ename='FORD')
""")

df_sql3 = pd.DataFrame(cur.fetchall(),columns = [ item[0] for item in cur.description ] )
df_sql3

,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7902,FORD,ANALYST,7566,3-DEC-81,3000,None,20


In [52]:
# multi row sub query example 

cur.execute(""" SELECT * FROM emp 
WHERE deptno IN (SELECT deptno FROM emp WHERE job='CLERK') 
AND job <> 'CLERK'; """)

df_sql4 = pd.DataFrame(cur.fetchall(),columns = columns )
df_sql4

,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7566,JONES,MANAGER,7839,2-APR-81,2975,None,20
1,7902,FORD,ANALYST,7566,3-DEC-81,3000,None,20


In [58]:
# Scaler Sub Query example

cur.execute(""" SELECT  deptno FROM emp WHERE ename='FORD'; """)

df_sql5 = pd.DataFrame(cur.fetchall(),columns = [item[0] for item in cur.description] )
df_sql5

,deptno
0,20


In [57]:
cur.description

(('deptno', None, None, None, None, None, None),)

In [47]:
columns = [ item[0] for item in cur.description ]
columns

['empno', 'ename', 'job', 'mgr', 'hiredate', 'sal', 'comm', 'deptno']

In [5]:
# Inline View Sub Query example

cur.execute(""" SELECT * FROM (SELECT * FROM emp ORDER BY sal DESC); """)

df_sql6 = pd.DataFrame(cur.fetchall(),columns = [ item[0] for item in cur.description ] )
df_sql6

,empno,ename,job,mgr,hiredate,sal,comm,deptno


In [39]:
# Co-related sub query examples

cur.execute(""" SELECT e.ename, e.deptno, (SELECT d.dname 
FROM dept d WHERE d.deptno = e.deptno) department FROM emp e ; """)

df_sql7 = pd.DataFrame(cur.fetchall(),columns = [ item[0] for item in cur.description ] )
df_sql7

OperationalError: no such column: e.ename

In [40]:
cur.execute("""select * from emp""").fetchall()

[(7369, 'SMITH', 'CLERK', 7902, '17-DEC-80', 800, None, 20),
 (7499, 'ALLEN', 'SALESMAN', 7698, '20-FEB-81', 1600, 300, 30),
 (7521, 'WARD', 'SALESMAN', 7698, '22-FEB-81', 1250, 500, 30),
 (7566, 'JONES', 'MANAGER', 7839, '2-APR-81', 2975, None, 20),
 (7654, 'MARTIN', 'SALESMAN', 7839, '2-APR-81', 1250, 1400, 30),
 (7902, 'FORD', 'ANALYST', 7566, '3-DEC-81', 3000, None, 20)]

# Analytical Funtions 
## COUNT(), SUM(),ROW_NUMBER(), RANK(), DENSE_RANK(), LEAD(), LAG()

In [27]:
# example for Rank() function
#Display all the employees details and display the rank of employees based on their salaries

cur.execute("""
Select ename,deptno,sal, rank() over(order by sal desc) as rank from emp""")

pd.DataFrame(cur.fetchall(),columns =[item[0] for item in cur.description])



,ename,deptno,sal,rank
0,FORD,20,3000,1
1,JONES,20,2975,2
2,ALLEN,30,1600,3
3,WARD,30,1250,4
4,MARTIN,30,1250,4
5,SMITH,20,800,6


In [3]:
cur.execute("""select * from emp""")
cur.fetchall()

[]

In [26]:
# Dense_rank() 

# Displaying difference between dense_rank and rank function
cur.execute("""
SELECT ename, deptno, sal, 
DENSE_RANK() OVER (ORDER BY sal DESC) dense_rank, 
RANK() OVER (ORDER BY sal DESC) rank
FROM emp; """)


pd.DataFrame(cur.fetchall(), columns = [ item[0] for item in cur.description])

,ename,deptno,sal,dense_rank,rank
0,FORD,20,3000,1,1
1,JONES,20,2975,2,2
2,ALLEN,30,1600,3,3
3,WARD,30,1250,4,4
4,MARTIN,30,1250,4,4
5,SMITH,20,800,5,6


In [30]:
# Row_number()

# Display top five highest salary of employee

cur.execute("""
SELECT empno, ename, sal, row_number 
FROM (SELECT empno, ename, sal, ROW_NUMBER( ) OVER(ORDER BY sal DESC) AS row_number FROM emp) 
WHERE row_number<=5; """)


pd.DataFrame(cur.fetchall(),columns = [item[0] for item in cur.description])

,empno,ename,sal,row_number
0,7902,FORD,3000,1
1,7566,JONES,2975,2
2,7499,ALLEN,1600,3
3,7521,WARD,1250,4
4,7654,MARTIN,1250,5


In [31]:
# LAG ()

cur.execute("""
SELECT ename, sal, LAG(sal,2,0) OVER (ORDER BY sal) lag
FROM emp; """)


pd.DataFrame(cur.fetchall(),columns = [item[0] for item in cur.description])

,ename,sal,lag
0,SMITH,800,0
1,WARD,1250,0
2,MARTIN,1250,800
3,ALLEN,1600,1250
4,JONES,2975,1250
5,FORD,3000,1600


In [33]:
# LEAD ()

cur.execute("""
SELECT ename, sal, LEAD(sal,2,0) OVER (ORDER BY sal) lead 
FROM emp; """)


pd.DataFrame(cur.fetchall(),columns = [item[0] for item in cur.description])

,ename,sal,lead
0,SMITH,800,1250
1,WARD,1250,1600
2,MARTIN,1250,2975
3,ALLEN,1600,3000
4,JONES,2975,0
5,FORD,3000,0


In [34]:
# group function 
# sum()

cur.execute("""
SELECT empno, ename, deptno, sal, SUM(sal) OVER(PARTITION BY deptno) FROM emp; """)

pd.DataFrame(cur.fetchall(),columns= [ item[0]for item in cur.description])

,empno,ename,deptno,sal,SUM(sal) OVER(PARTITION BY deptno)
0,7369,SMITH,20,800,6775
1,7566,JONES,20,2975,6775
2,7902,FORD,20,3000,6775
3,7499,ALLEN,30,1600,4100
4,7521,WARD,30,1250,4100
5,7654,MARTIN,30,1250,4100


In [42]:
# group function 
# sum()

cur.execute("""
SELECT empno, ename, deptno, sal, count(sal) OVER(PARTITION BY deptno ORDER BY deptno) FROM emp; """)

pd.DataFrame(cur.fetchall(),columns= [ item[0]for item in cur.description])

,empno,ename,deptno,sal,count(sal) OVER(PARTITION BY deptno ORDER BY deptno)
0,7369,SMITH,20,800,3
1,7566,JONES,20,2975,3
2,7902,FORD,20,3000,3
3,7499,ALLEN,30,1600,3
4,7521,WARD,30,1250,3
5,7654,MARTIN,30,1250,3


In [44]:
cur.execute("""
select * from emp""").fetchall()

[(7369, 'SMITH', 'CLERK', 7902, '17-DEC-80', 800, None, 20),
 (7499, 'ALLEN', 'SALESMAN', 7698, '20-FEB-81', 1600, 300, 30),
 (7521, 'WARD', 'SALESMAN', 7698, '22-FEB-81', 1250, 500, 30),
 (7566, 'JONES', 'MANAGER', 7839, '2-APR-81', 2975, None, 20),
 (7654, 'MARTIN', 'SALESMAN', 7839, '2-APR-81', 1250, 1400, 30),
 (7902, 'FORD', 'ANALYST', 7566, '3-DEC-81', 3000, None, 20)]

In [45]:
# create simple table emp_company  using simple syntax
cur.execute("""
CREATE TABLE emp_company
(
ename varchar(10) not null,
cname varchar(20),
salary Number(7,2),
jdate date
);
""")

In [46]:
cur.execute("""
select * from emp_company""").fetchall()

[]

In [47]:
# create simple table emp_company2 with help of emp table
cur.execute("""
CREATE TABLE emp_company2 As 
Select * from emp where 1 =2;
""")

In [48]:
# check emp_company2

cur.execute("""
select * from emp_company2""").fetchall()

[]

In [49]:
# create simple table emp_company1 with help of emp table
# condition deptno =2 
cur.execute("""
CREATE TABLE emp_company1 As 
Select * from emp where deptno = 20;
""")

In [50]:
# check emp_company1

cur.execute("""
select * from emp_company1""").fetchall()

[(7369, 'SMITH', 'CLERK', 7902, '17-DEC-80', 800, None, 20),
 (7566, 'JONES', 'MANAGER', 7839, '2-APR-81', 2975, None, 20),
 (7902, 'FORD', 'ANALYST', 7566, '3-DEC-81', 3000, None, 20)]

In [52]:
# create emp_shift table by using table create syntax


cur.execute("""
CREATE TABLE emp_shift
(
ename varchar(10) not null,
salary Number(7,2)
);
""")

In [55]:
# 2. insert with selected columns name from another table  and condition

cur.execute("""
INSERT INTO emp_shift (ename,salary) 
SELECT ename,sal FROM emp_company1
WHERE ename = 'SMITH';
""")





